In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import pickle
import tensorflow as tf

import fasttext
import fasttext.util

# Hyperlex load (i dont use yet)


In [5]:
file = "../Data/hyperlex-data/hyperlex-all.txt"

results = []
with open(file) as f:
    line = f.readline()
    while line:
        results.append(line.strip().split(" "))
        line = f.readline()
f.close()

HyperLex = pd.DataFrame(results)
HyperLex.columns = HyperLex.iloc[0]
HyperLex = HyperLex.iloc[1:].reset_index(drop=True)

HyperLex.head()


,WORD1,WORD2,POS,TYPE,AVG_SCORE,AVG_SCORE_0_10,STD,SCORES..,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,conflict,disagreement,N,r-hyp-1,5.20,8.67,1.25,5,6,6,...,6,5,2,6,4,6,None,None,None,None
1,advance,take,V,no-rel,1.42,2.37,1.66,3,0,3,...,0,5,0,0,1,2,3,0,None,None
2,trail,follow,V,hyp-2,4.31,7.18,1.86,4,4,6,...,6,6,6,3,0,4,3,6,6,None
3,mason,worker,N,hyp-3,4.50,7.5,1.76,6,6,4,...,6,3,6,3,5,0,5,6,None,None
4,aura,light,N,hyp-1,3.69,6.15,1.86,4,5,3,...,6,0,3,6,3,0,4,5,5,None


# load baroni dataset

In [6]:
neg_file = "../Data_shared/eacl2012-data/negative-examples.txtinput"
pos_file = "../Data_shared/eacl2012-data/positive-examples.txtinput"
filenames = ["neg_file", "pos_file"]


for i, file in enumerate([neg_file, pos_file]):
    globals()['results_{}'.format(filenames[i])] = []
    
    with open(file) as f:
        line = f.readline()
        while line:
            globals()['results_{}'.format(filenames[i])].append(line.replace("-n", "").replace("\n", "").strip("").split("\t"))
            line = f.readline()
    f.close()


In [8]:
baroni = sum(results_neg_file, []) + sum(results_pos_file, [])
set_baroni = set(baroni)

print("the length of all words in baroni: ", len(baroni))
print("number of unique in baroni: ", len(set(set_baroni)))


the length of all words in baroni:  5540
number of unique in baroni:  1478


# load wikipedia data(sub)set 
Here I take a subset from the wikipedia data and create a method to substract a set of context vectors.

I found the most common word in the 4 text is "church", i use that for further testing.

In [11]:
# ran this on the workstation and copied it into the notebook
import ast

with open('../Data_shared/wiki_subset.txt') as file:
    data = file.read()

wikidata = ast.literal_eval(data)


### Pre processing the wikipedia dataset

In [12]:
import re
import numpy as np  

def text_preprocessing(
    text:list,
    punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_“~''',
    stop_words=['and', 'a', 'is', 'the', 'in', 'be', 'will']
    )->list:
    """
    A method to preproces text
    """
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")

    # Removing words that have numbers in them
    text = re.sub(r'\w*\d\w*', '', text)

    # Removing digits
    text = re.sub(r'[0-9]+', '', text)

    # Cleaning the whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Setting every word to lower
    text = text.lower()

    # Converting all our text to a list 
    text = text.split(' ')

    # Droping empty strings
    text = [x for x in text if x!='']

    # Droping stop words
#     text = [x for x in text if x not in stop_words]

    return text

### apply preprocessing to wikidata

In [13]:
# Cleaning the text
texts = [x for x in wikidata['text']]

wiki_all_text = []

for text in tqdm(texts):
    # Appending to the all text list
    text = text_preprocessing(text)
    wiki_all_text += text
    

100%|██████████| 10000/10000 [00:19<00:00, 507.45it/s]


In [14]:
wiki_all_text[:10]

['yangliuqing',
 'is',
 'a',
 'market',
 'town',
 'in',
 'xiqing',
 'district',
 'in',
 'the']

### Creating a context dictionairy from wiki subset 

In [15]:
# Defining the window for context
window = 5

# Creating a dictionary entry for each word in the texts
context_dict = { i : list() for i in set(wiki_all_text)}


for i, word in tqdm(enumerate(wiki_all_text)):
    for w in range(window):
        # Getting the context that is ahead by *window* words
        
        if i + 1 + w < len(wiki_all_text):
            context_dict[word].append(wiki_all_text[(i + 1 + w)]) 
        # Getting the context that is behind by *window* words    
        if i - w - 1 >= 0:
            context_dict[word].append(wiki_all_text[(i - w - 1)])


4556903it [00:31, 146322.34it/s]


# use fasttext to get word vectors


In [38]:
# get pre trained fastext model code is now replaced with a load file

# ft = fasttext.load_model('./data/cc.en.300.bin')
# ft.get_dimension()
# fasttext.util.reduce_model(ft, 100)
# ft.get_dimension()

# ft.save_model("ft_reduced_100.bin")

In [21]:
ft = fasttext.load_model("../Data/ft_reduced_100.bin")

# EMPIRICAL COVARIANCES
the actual implementation of the equation

In [18]:
# Extract words that appear in both baroni as in wikidata subset
combined_set = set(wiki_all_text)&set(set_baroni)
print("unique words in baroni: ", len(set(set_baroni)))
print("words that are represented by the wiki subset: ", len(combined_set))

combined_set

unique words in baroni:  1478
words that are represented by the wiki subset:  1448


In [53]:
total = np.zeros((100,100))

test_words = ['church', 
              'building']

covariance = {}

for word in tqdm(combined_set):
    for c_word in context_dict[word]:
        w = ft.get_word_vector(word)
        total += np.outer((ft.get_word_vector(c_word) - w), (ft.get_word_vector(c_word) - w))
    
    covariance[word] = (total / (len(context_dict[word]) * window))


#### create a combined set with enough context to keep from getting PositiveDefinite() error

In [90]:
combined_set_context_len = {}

for i in combined_set:
    combined_set_context_len[i] = len(context_dict[i])

# coriander triggered positive definite warning context of coriander is 20 words
combined_set_context_len['coriander']

combined_set_30plus = [x for x , key in combined_set_context_len.items() if key > 30]
len(combined_set_30plus)

1318

In [24]:


mean1 = ft.get_word_vector('church')
covariance_matrix1 = covariance["church"]
# multiplyDiagonal(covariance_matrix1, 0.5)

mean2 = ft.get_word_vector('building')
covariance_matrix2 = covariance["building"]
# multiplyDiagonal(covariance_matrix2, 0.5)

mean1 = torch.from_numpy(mean1)
covariance_matrix1 = torch.from_numpy(covariance_matrix1)

mean2 = torch.from_numpy(mean2)
covariance_matrix2 = torch.from_numpy(covariance_matrix2)


p = torch.distributions.multivariate_normal.MultivariateNormal(mean1, covariance_matrix=covariance_matrix1)
q = torch.distributions.multivariate_normal.MultivariateNormal(mean2, covariance_matrix=covariance_matrix2)



torch.distributions.kl.kl_divergence(p, q)


tensor(19.7034, dtype=torch.float64)

In [36]:
#  In practice, it is also necessary to add a small ridge term 
#  δ > 0 to the diagonal of the matrix to regularize and avoid 
#  numerical problems when inverting - vilnis mccalumn

def addDiagonal(matrix, x):
    assert x < 1, f"x greater than 0 expected, got: {x}"
    
    for i in range(len(matrix)):
        matrix[i][i] = matrix[i][i] + x
    
    return matrix


# a = np.arange(9).reshape(3,3)
# print(a)
# addDiagonal(a, 0.3)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

## Baroni test set 

- count word occurence of baroni in wiki

In [91]:
# create a baroni pos and neg dataset that only contains words from the combined word collection
# so ones that are found in the wiki dataset 

results_neg_file
results_pos_file
baroni_pos_subset = [x for x in results_pos_file if x[0] in combined_set_30plus and x[1] in combined_set_30plus]
baroni_neg_subset = [x for x in results_neg_file if x[0] in combined_set_30plus and x[1] in combined_set_30plus]

print("baroni pos length: ", len(results_pos_file))
print("baroni pos subset length: ",len(baroni_pos_subset))
print("baroni neg length: ", len(results_neg_file))
print("baroni neg subset length: ",len(baroni_neg_subset))

baroni pos length:  1385
baroni pos subset length:  1172
baroni neg length:  1385
baroni neg subset length:  1179


In [97]:
baroni_subset_label = []

for i in baroni_pos_subset:
    baroni_subset_label.append([i, 1])

for i in baroni_neg_subset:
    baroni_subset_label.append([i, 0])

df = pd.DataFrame(baroni_subset_label, columns =['Wordpair', 'True label'])
df

,Wordpair,True label
0,"[abstraction, concept]",1
1,"[accusation, statement]",1
2,"[ache, pain]",1
3,"[acid, chemical]",1
4,"[acoustics, discipline]",1
...,...,...
2346,"[writer, dramatist]",0
2347,"[writer, poet]",0
2348,"[yellow, protein]",0
2349,"[yesterday, discipline]",0


### KL divergence

In [101]:
baroni_subset_kl = []

for wordpair in (baroni_pos_subset + baroni_neg_subset):
    mean1 = torch.from_numpy(ft.get_word_vector(wordpair[0]))
    covariance_matrix1 = torch.from_numpy(covariance[wordpair[0]])
    mean2 = torch.from_numpy(ft.get_word_vector(wordpair[1]))
    covariance_matrix2 = torch.from_numpy(covariance[wordpair[1]])
    
    p = torch.distributions.multivariate_normal.MultivariateNormal(mean1, covariance_matrix=covariance_matrix1)
    q = torch.distributions.multivariate_normal.MultivariateNormal(mean2, covariance_matrix=covariance_matrix2)

    baroni_subset_kl.append(float(torch.distributions.kl.kl_divergence(p, q)))


In [130]:
df['KL score'] = baroni_subset_kl


### COS similairity

In [145]:
from numpy.linalg import norm
from scipy.spatial import distance


In [116]:
def cosine_similarity(a, b):
    nominator = np.dot(a, b)
    
    a_norm = np.sqrt(np.sum(a**2))
    b_norm = np.sqrt(np.sum(b**2))
    
    denominator = a_norm * b_norm
    
    cosine_similarity = nominator / denominator
    
    return cosine_similarity


In [123]:
covariance["church"].flatten()

10000

In [138]:
baroni_subset_cos = []

for wordpair in (baroni_pos_subset + baroni_neg_subset):
    A = ft.get_word_vector(wordpair[0])
    B = ft.get_word_vector(wordpair[1])
    baroni_subset_cos.append(cosine_similarity(A, B))
    

In [141]:
df['COS score'] = baroni_subset_cos


In [144]:
df

,Wordpair,True label,KL score,COS score
0,"[abstraction, concept]",1,3677.993391,0.469579
1,"[accusation, statement]",1,33120.671239,0.614132
2,"[ache, pain]",1,310.987227,0.792009
3,"[acid, chemical]",1,12.417440,0.668829
4,"[acoustics, discipline]",1,13.651013,0.216910
...,...,...,...,...
2346,"[writer, dramatist]",0,76.266101,0.694186
2347,"[writer, poet]",0,28.792145,0.798863
2348,"[yellow, protein]",0,20.560677,0.333175
2349,"[yesterday, discipline]",0,37.131087,-0.018511


In [148]:
import numpy as np
from sklearn.metrics import average_precision_score

print("COS AP: ", average_precision_score(df["True label"], df["COS score"]))
print("KL AP: ", average_precision_score(df["True label"], df["KL score"]))

COS AP:  0.7310373436269322
KL AP:  0.5979891637917796


In [150]:
from collections import Counter
wiki_count = Counter(tqdm(wiki_all_text))


100%|██████████| 4556903/4556903 [00:04<00:00, 1110577.82it/s]


In [151]:
wiki_count

Counter({'yangliuqing': 8,
         'is': 39500,
         'a': 91181,
         'market': 560,
         'town': 1709,
         'in': 135269,
         'xiqing': 2,
         'district': 3305,
         'the': 312013,
         'western': 1427,
         'suburbs': 71,
         'of': 160499,
         'tianjin': 23,
         'peoples': 524,
         'republic': 1112,
         'china': 1022,
         'despite': 834,
         'its': 8690,
         'relatively': 255,
         'small': 1503,
         'size': 492,
         'it': 19342,
         'has': 9452,
         'been': 6368,
         'named': 1966,
         'since': 2683,
         'famous': 487,
         'historical': 720,
         'and': 127523,
         'cultural': 491,
         'towns': 398,
         'best': 2002,
         'known': 3252,
         'for': 38520,
         'creating': 228,
         'nianhua': 2,
         'or': 10404,
         'more': 4473,
         'than': 3274,
         'years': 4409,
         'effect': 446,
         'speciali

In [162]:
wiki_all_text = []
chunk = int(len(texts)/10)
import time

for i in range(10):
    for text in tqdm(texts[i:(i+1)*chunk]):
        # Appending preprocessed text to the "all text" list
        text = text_preprocessing(text)
        wiki_all_text += text
    
    time.sleep(1)
    print(f"chunk {i}")

100%|██████████| 1000/1000 [00:02<00:00, 373.14it/s]


chunk 0


100%|██████████| 1999/1999 [00:04<00:00, 459.18it/s]


chunk 1


 33%|███▎      | 982/2998 [00:02<00:05, 348.92it/s]


KeyboardInterrupt: 